<a href="https://colab.research.google.com/github/carsondenison/proximal-policy-optimization/blob/main/PPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet "torch" "pytorch-lightning" "gym"

     |████████████████████████████████| 525 kB 5.2 MB/s 
     |████████████████████████████████| 829 kB 32.3 MB/s 
     |████████████████████████████████| 596 kB 46.1 MB/s 
     |████████████████████████████████| 332 kB 50.8 MB/s 
     |████████████████████████████████| 132 kB 50.6 MB/s 
     |████████████████████████████████| 1.1 MB 34.2 MB/s 
     |████████████████████████████████| 160 kB 40.7 MB/s 
     |████████████████████████████████| 192 kB 46.2 MB/s 
     |████████████████████████████████| 271 kB 44.9 MB/s 


Step 0: Import the libraries we'll need

In [2]:
import random
from typing import List, Tuple, Iterable
from collections import namedtuple, deque

import gym
import numpy as np
import torch
import torch.nn as nn
from torch import Tensor
# import pytorch_lightning as pl

# Implements: https://arxiv.org/pdf/1707.06347.pdf

Step 1: Create a dataset to store experiences

In [3]:
Experience = namedtuple('Experience', 'state action reward done new_state')

class ReplayBuffer():
    '''
        Buffer to hold Experiences for training
    '''
    def __init__(self):
        self.buffer:List[Experience] = []
    
    def append(self, x):
        self.buffer.append(x)
    
    def clear(self):
        self.buffer = []

    def to_batch(self) -> Tuple[Tensor, Tensor, Tensor, Tensor, Tensor]:
        states, actions, rewards, dones, new_states = zip(*self.buffer)
        states = torch.tensor(states, dtype=torch.float32)
        actions = torch.tensor(actions, dtype=torch.int64)
        rewards = torch.tensor(rewards, dtype=torch.float32)
        dones = torch.tensor(dones, dtype=torch.bool)
        new_states = torch.tensor(new_states, dtype=torch.float32)
        return states, actions, rewards, dones, new_states

Step 2: Create an actor that can interact with the environment

In [4]:
class Actor():
    '''
        Class which can interact with the environment
    '''
    def __init__(self, env:gym.Env, replay_buffer:ReplayBuffer, pi:nn.Module):
        self.env = env
        self.buffer = buffer
        self.pi = pi
        self.state = self.env.reset() # self.state is a numpy array

    def get_action(self) -> int:
        '''
            Samples the policy to get an action given self.state
        '''
        pi_logits = self.pi(torch.tensor(self.state))
        policy = torch.distributions.categorical.Categorical(logits=pi_logits)
        action = policy.sample()
        return action.item()

    @torch.no_grad()
    def play_step(self) -> None:
        '''
            Play one step of the environment, and add it to the buffer
        '''
        action = self.get_action()
        new_state, reward, done, _ = self.env.step(action)
        exp = Experience(self.state, action, reward, done, new_state)
        self.buffer.append(exp)
        self.state = new_state
        if done:
            self.state = self.env.reset()
        return done


Step 3: Define the neural network architecture for policy and advantage

In [5]:
class MLP(nn.Module):
    '''
        Simple MLP, as described in https://arxiv.org/pdf/1707.06347.pdf
    '''
    def __init__(self, in_size, out_size, hidden_size=64):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, out_size),
        )

    def forward(self, state):
        return self.net(state.float())

Step 4: Define an advantage estimator function. This is built from a value network and a reward_to_go calculator



In [6]:
def reward_to_go(rewards:Tensor, gamma:float) -> Tensor:
    '''
        Calculates the rewards_to_go for a trajectory

        Args:
            rewards: (T, 1) tensor of rewards for each step
            gamma: discount factor for each step
        Returns:
            rewards_to_go: Discounted reward_to_go for each reward in rewards
    '''
    if len(rewards) <= 1:
        return rewards
    rewards_to_go = torch.zeros_like(rewards, dtype=torch.float32)
    for i in range(-1, -len(rewards)-1, -1):
        rewards_to_go[i] = rewards[i] + gamma * rewards_to_go[i + 1]
    return rewards_to_go  

# quick tests for reward_to_go
rtg = reward_to_go(torch.tensor([1, 1, 1,]), 1)
assert torch.all(torch.eq(rtg, torch.tensor([3, 2, 1]))), str(rtg)
rtg2 = reward_to_go(torch.tensor([1, 1, 1]), 0.5)
assert torch.all(torch.eq(rtg2, torch.tensor([1.75, 1.5, 1]))), str(rtg2)
rtg3 = reward_to_go(torch.tensor([]), 1)
assert torch.all(torch.eq(rtg3, torch.tensor([])))
rtg4 = reward_to_go(torch.tensor([1]), 0.5)
assert torch.eq(rtg4, torch.tensor(1)), str(rtg4)

In [7]:
class FakeValue(nn.Module):
    '''
        torch.nn.Module that returns the identiy. Useful for testing
    '''
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x

def estimate_advantage(states:Tensor, rewards:Tensor, value_net:nn.Module, gamma:float, final_state:Tensor=None) -> Tensor:
    '''
        Compute advantage estimate for each step in a trajectory

        Args:
            rewards: Reward given by each step in the trajectory. Size (T, 1)
            states: State vectors for trajectory. Size (T, state_size)
            v_net: Trainable network which predicts the value V(s) of a state
            gamma: Discount factor. Assume lambda = 1 from GAE-Lambda
            final_state: if given final_state, then non-terminal so use v(final_state). If not given, assume terminal, v(s_T) = 0
        Returns:
            advantages: The estimated advantage for each step in the trajectory
    '''
    T = len(rewards) + 1

    # Is reward-to-go correct when last state is terminal?
    assert len(rewards) == len(states), 'len(rewards) doesnt match len(states)'
    values = value_net(states) # Shape (T, 1)
    # If we're given a final state, non-terminal so compute its value
    if final_state is not None:
        final_value = value_net(final_state)
    # else, terminal state, gets value of 0
    else:
        final_value = 0
    rewards_to_go = reward_to_go(rewards, gamma)
    discounted_final_values = [final_value * (gamma) ** n for n in reversed(range(1, T))]
    discounted_final_values = torch.tensor(discounted_final_values)
    advantages = rewards_to_go - values + discounted_final_values
    return advantages

fake_value_net = FakeValue()
r = torch.tensor([1,1])
s = torch.tensor([1,1])

# Test that discounting of values works with gamma = 1, done = True
adv = estimate_advantage(r, s, fake_value_net, 1)
assert torch.allclose(adv, torch.tensor([1,0], dtype=torch.float32)), adv
# Test that discounting of values works with gamma = 0.3, done = True
adv2 = estimate_advantage(r, s, fake_value_net, 0.3)
assert torch.allclose(adv2, torch.tensor([0.3,0], dtype=torch.float32)), adv2

final_state = torch.tensor([2])
# Test that this works with a final_state
adv3 = estimate_advantage(r, s, fake_value_net, 0.5, final_state)
assert torch.allclose(adv3, torch.tensor([1,1], dtype=torch.float32)), adv3

Step 5: Define the clipped loss function

In [8]:
def clipped_loss(states:Tensor, actions:Tensor, advantages:Tensor, pi_old:Tensor, pi_net:nn.Module, epsilon=0.2) -> Tensor:
    '''
        PPO Clipped Loss

        Args:
            states: The states from a given trajectory T
            advantages: Advantage estimates for T, based on GAE-Lambda
            pi_old: Probability distribution for pi(a|s) used to generate the trajectory
            pi_net: Most up to date policy network
            epsilon: Clipping hyperparameter for loss. See page 3 - https://arxiv.org/pdf/1707.06347.pdf
        Returns:
            loss: L_clip used to optimize the policy network
    '''
    pi = pi_net(states).gather(1, actions[:, None])
    ratio = torch.div(pi, pi_old)
    unclipped = ratio * advantages
    clipped = torch.clamp(ratio, 1-epsilon, 1+epsilon) * advantages
    elementwise_mins = torch.minimum(unclipped, clipped)
    loss = torch.mean(elementwise_mins)
    return loss

Step 6: Main Training Loop


In [14]:
# Set up the environment
env = gym.make('CartPole-v0')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

# Build the neural networks and optimizers
pi = MLP(state_size, action_size)
v = MLP(state_size, 1)

pi_optimizer = torch.optim.Adam(pi.parameters(), lr=5e-3)
v_optimizer = torch.optim.Adam(v.parameters(), lr=5e-3)
v_loss_fn = nn.MSELoss()

# Set up the buffer and actor
buffer = ReplayBuffer()
actor = Actor(env, buffer, pi)

# Hyperparameters
total_epochs = 301
longest_episode_length = 0
gamma = 0.99
K_pi = 20
K_v = 20

for epoch in range(total_epochs):
    # Play up to 200 steps, stopping if terminal state reached
    # In a real implementation, this would be done many times in parallel with many actors
    # That serves to reduce variance and overfitting to any individual trajectory
    # We intentionally take only one trajectory at a time to make things clearer
    for episode_length in range(200):
        done = actor.play_step()
        if done:
            break

    # Log epoch information to the console
    if epoch % 100 == 0:
        print(f'epoch: {epoch}')
    if episode_length >= longest_episode_length:
        longest_episode_length = episode_length
        print(f'episode: {epoch} length: {longest_episode_length}')

    # Unpack replay buffer into arrays and reset the buffer
    states, actions, rewards, _, new_states = buffer.to_batch()
    buffer.clear()

    # Compute the information we'll need to feed the two loss functions
    if done:
        final_state = None
    else:
        final_state = new_states[-1]
    advantages = estimate_advantage(states, rewards, v, gamma, final_state).detach()
    pi_old = pi(states).gather(1, actions[:, None]).detach()
    rtg = reward_to_go(rewards, gamma)

    # Update policy network for K steps
    # A full implementation have a KL divergence check here to stop updating policy network and get another trajectory when the policy changes too much
    for _ in range(K_pi):
        pi_loss = clipped_loss(states, actions, advantages, pi_old, pi)
        pi_optimizer.zero_grad()
        pi_loss.backward()
        pi_optimizer.step()

    # Update Value Network V(s_t) should be reward_to_go_t. MSE Loss
    for _ in range(K_v):
        values = v(states).squeeze()
        v_loss = v_loss_fn(values, rtg)
        v_optimizer.zero_grad()
        v_loss.backward()
        v_optimizer.step()

epoch: 0
episode: 0 length: 53
episode: 4 length: 72
episode: 5 length: 76
episode: 53 length: 121
episode: 54 length: 199
episode: 55 length: 199
episode: 56 length: 199
episode: 59 length: 199
episode: 69 length: 199
episode: 94 length: 199
epoch: 100
episode: 111 length: 199
episode: 122 length: 199
episode: 125 length: 199
episode: 128 length: 199
episode: 129 length: 199
episode: 133 length: 199
episode: 136 length: 199
episode: 138 length: 199
episode: 142 length: 199
episode: 144 length: 199
episode: 161 length: 199
episode: 164 length: 199
episode: 165 length: 199
episode: 167 length: 199
episode: 169 length: 199
episode: 170 length: 199
episode: 171 length: 199
episode: 174 length: 199
episode: 181 length: 199
episode: 195 length: 199
epoch: 200
epoch: 300
